### Mounting drive on Colab

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


###Libraries

In [ ]:
!pip install tensorflow_addons

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
The folder you are executing pip from can no longer be found.


In [ ]:
import os
import cv2
import os
import h5py
import numpy as np
import pickle
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow_addons.metrics import HammingLoss
from tensorflow_addons.metrics import F1Score
os.chdir('/gdrive/My Drive/')
from numpy import savez_compressed
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras import layers, models
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

## Number of Neurons per hidden layer ratio to input layer (1/3, 1/2, 2/3, 3/4, 1)

### For Imagenet as a source

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()


pre_trained_model = InceptionV3(input_shape = (96, 96, 3), 
                                include_top = False, 
                                weights = 'None')
#for layer in pre_trained_model.layers:
#  layer.trainable = False

In [ ]:
def compile_model(rate):
  from tensorflow.keras.optimizers import RMSprop

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(pre_trained_model.output)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(2000, activation='relu')(x)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(int(2000*rate), activation='relu')(x)
  # Add a fully connected layer with 2000 hidden units and ReLU activation
  x = layers.Dense(int(2000 * (rate) * (rate)), activation='relu')(x)                  
  # Add a final sigmoid layer for classification
  x = layers.Dense  (15, activation='softmax')(x)           

  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc', tf.keras.metrics.AUC(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), HammingLoss(threshold=0.5, mode='multiclass'), F1Score(num_classes= 15, threshold=0.5)])
  return model

In [ ]:
data = np.load('Xtrain.npy', allow_pickle=True)
labels = np.load('ytrain.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [ ]:
y_train = to_categorical(y_train, 15)
y_val = to_categorical(y_val, 15)
y_test = to_categorical(y_test, 15)

In [ ]:
for i in [1/3, 1/2, 2/3, 3/4, 1]:
  print("******************* Training model for ratio of", i, "************************")
  imagenet_inception_v3 = compile_model(i)
  history = imagenet_inception_v3.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
  print("******************* Model evaluation for a ratio of", i, "*************************")
  eval = imagenet_inception_v3.evaluate(X_test, y_test)
  print("******************* Train time is", sum(cb.logs), "***********")
  print("f1 score is", sum(eval[6])/15)

******************* Training model for ratio of 0.3333333333333333 ************************


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 32s 129ms/step - loss: 9.1894 - acc: 0.1069 - auc_2: 0.5530 - recall_2: 0.0689 - precision_2: 0.1147 - hamming_loss: 0.9311 - f1_score: 0.0850 - val_loss: 3.4240 - val_acc: 0.1263 - val_auc_2: 0.5989 - val_recall_2: 0.0587 - val_precision_2: 0.2189 - val_hamming_loss: 0.9413 - val_f1_score: 0.0682
Epoch 2/10
200/200 [==============================] - 23s 117ms/step - loss: 3.1209 - acc: 0.1655 - auc_2: 0.6346 - recall_2: 0.0629 - precision_2: 0.2997 - hamming_loss: 0.9371 - f1_score: 0.1000 - val_loss: 2.7242 - val_acc: 0.2127 - val_auc_2: 0.6749 - val_recall_2: 0.0663 - val_precision_2: 0.4482 - val_hamming_loss: 0.9337 - val_f1_score: 0.0988
Epoch 3/10
200/200 [==============================] - 23s 117ms/step - loss: 2.6048 - acc: 0.2289 - auc_2: 0.7030 - recall_2: 0.0822 - precision_2: 0.4745 - hamming_loss: 0.9178 - f1_score: 0.1337 - val_loss: 2.7033 - val_acc: 0.2210 - val_auc_2: 0.6884 - val_recall_2: 0.0840 - val_precision_2